---
# Projectr : 하루시작 지하철 혼잡도 분석
### Description : 
- <a><span style = "color: #FFBE98">**오늘의 주요 키워드 및 뉴스 추천하기**</a>
    
### Author : Zen Den
### Date : 2024. 06. 11. (Tue) ~
### Detail : 
### Update: 
- 2024.06.11. (Tue) K.Zen : 
  <a href = "https://news.naver.com/main/ranking/popularDay.naver">
    <span style = "color: #F7CAC9">
      <b>NAVER 랭킹뉴스</b>
  </a>
  에서 언론사별 5개씩 가져오기
  <span style = "color: #FFBE98">
    <b>(언론사명, Title, Link)</b>
  </span>
  <br><br>

- 2024.06.12. (Wed) K.Zen :
  - Selenium Library를 이용한 크롤링 과정 중 ScreenShot 촬영을 백그라운드로 실행하기 <br>
    [참고자료]
      <a href = "https://co-de.tistory.com/21">
        <span style = "color: #F7CAC9">
          <b>[selenium]</b>
          손쉽게 브라우저 자동 캡쳐 기능 만들기
      </a>
  <br><br>

- 2024.06.13. (Thu) K.Zen :
  - <span style = "color: cyan">
      <b>Deep Learning(TF-IDF 벡터화, 코사인 유사도 계산)</b>
      을 활용한 뉴스 추천하기
    </span>

---
# Analyse News Articles by media outlet to recommend <br> the top keywords and news of the day

## Import Library Package

### Basic

In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, warnings; warnings.filterwarnings('ignore')

### File System

In [2]:
import os; from pathlib import Path
from datetime import datetime

### Crawling

In [3]:
import requests # 인터넷에서 Data를 가져오기 위한 Library (웹페이지에 접속하고 HTML 코드를 가져오기 위해 사용)
from bs4 import BeautifulSoup # 웹 페이지 내용을 분석하기 위한 Library (가져온 HTML 코드에서 우리가 필요한 정보를 추출하기 위해 사용)
import time # 대기 시간을 추가하기 위한 Library (요청 사이에 랜덤한 시간을 기다리기 위해 사용)
import random # Random한 대기 시간을 만들기 위한 Library
from tqdm import tqdm # Crawling 진행 상황을 체크하기 위한 Module (진행 상황을 시각적으로 보여주기 위해 사용)

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

### Bag of Words (BoW)

In [4]:
from konlpy.tag import Okt
import nltk # Natural Language Toolkit (자연어 처리를 위해 사용)
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter # 단어의 빈도를 계산하기 위해 사용

### Deep Learning
- TF-IDF 벡터화와 코사인 유사도 계산을 위해 사용

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# NAVER News - 언론사별 랭킹뉴스 Crawling

## 언론사, Title, Link 추출하기

In [6]:
# 뉴스 Crawling
def get_news_links_by_press (url) :
  """
    headers:
    - 나는 bot이 아니고 사람임을 증명하는 부분이다.
      - 사용하지 않을 시 언론사에서 웹크롤링을 막을 수 있으니 주의할 것!
  """
  headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36"
  }
  response = requests.get(url, headers = headers) # url(페이지)에 접속
  soup = BeautifulSoup(response.content, 'html.parser') # HTML 코드를 파싱(분석)하여 soup 객체에 저장
  
  press_data = {}
  press_sections = soup.select('.rankingnews_box')
  
  for press_section in tqdm(press_sections, desc = "언론사별 뉴스 Crawling 진행 중") :
    press_name = press_section.select_one('.rankingnews_name').get_text(strip = True)
    news_links = set()  # 중복 제거를 위한 set 사용
    for item in press_section.select('li a') :
      title = item.get_text(strip = True)
      link = item['href']
      if title and link and "동영상" not in title :  # Title이 존재하고 "동영상"이 포함되지 않은 경우에만 추가
        news_links.add((title, link))
    press_data[press_name] = list(news_links)[:5]  # 다시 list로 변환 후 상위 5개만 저장
    
    # 각 언론사별 뉴스 Crawling 후 대기 시간 추가
    time.sleep(random.uniform(0.5, 2.0))
  
  return press_data

In [7]:
base_url = 'https://news.naver.com/main/ranking/popularDay.naver'
press_news_data = get_news_links_by_press(base_url)

# 뉴스 DataFrame 생성
news_list = []
for press_name, news_data in press_news_data.items() :
  for title, link in news_data :
    news_list.append([press_name, title, link])
df = pd.DataFrame(news_list, columns = ['Press', 'Title', 'Link'])

df.head()

언론사별 뉴스 Crawling: 100%|██████████| 82/82 [03:57<00:00,  2.90s/it]


,Press,Title,Link
0,YTN,"허위 서류로 수십억 나랏돈 횡령...""前 고위공무원도 가담""",https://n.news.naver.com/article/052/000204754...
1,YTN,"[단독] ""문제없다""던 인천공항...취재 시작되자 '일사불란'",https://n.news.naver.com/article/052/000204752...
2,YTN,퇴근길 만취 운전 사고나서 '덜미'...바퀴 터진 택배차량 화재,https://n.news.naver.com/article/052/000204753...
3,YTN,아파트 외벽에 뜬금없이 '김대중'...입주민들 황당,https://n.news.naver.com/article/052/000204757...
4,YTN,고물가 속 등장한 6천 원대 팥빙수...또 여기? [앵커리포트],https://n.news.naver.com/article/052/000204756...


In [17]:
# 오늘 날짜 가져오기
today = datetime.now().strftime('%Y%m%d')

df.to_csv(f"Data/NAVER_News_List_{today}.csv", index = False)

---

## 본문 수집하기

In [12]:
# Chrome Browser와 Chrome Driver Version 확인 및 WebDriver 객체 생성
chrome_options = webdriver.ChromeOptions()
# ******************************************************
chrome_options.add_argument('headless') # Run chrome browser in the background
chrome_options.add_argument('window-size = 1920x1080')  # Chrome Browser Window Size
# ******************************************************
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()), options = chrome_options)

# 1. 언론사 목록 가져오기
press_list = list(df['Press'].unique())  # random.sample()은 Sequence Type의 Data만 지원하기 때문에 df.Press.unique()의 결과를 List로 변환해야 한다.

# 2. 랜덤으로 10개의 언론사 선택
selected_press = random.sample(press_list, 10)

# # 3. 선택된 언론사에서 Random으로 1~5개의 Link 추출
links = []
for press in selected_press :
    links.extend(df[df['Press'] == press].sample(random.randint(1, 5))['Link'].tolist())

# tqdm Module을 이용하여 진행 상황 표시
pbar = tqdm(links, desc = "뉴스 기사 본문 Crawling 진행 중", unit = "Link")

# 뉴스 기사 Link를 순회하며 기사 내용 추출
articles = []
for link in pbar :
    driver.get(link)

    # **************************************************************************
    # 기본 Path 설정
    today = datetime.now().strftime('%Y%m%d')   # 오늘 날짜 가져오기
    base_path = f"Data/ScreenShot/By_Press/{today}"

    ## By_Press 폴더 생성
    os.makedirs(base_path, exist_ok = True)

    # 동일한 By_Press/today Folder가 존재하는지 확인하고, 존재한다면 하위에 새로운 Folder 생성
    new_path = base_path
    while os.path.exists(new_path) :
        new_path = os.path.join(base_path, str(len(os.listdir(base_path)) + 1)) # index는 폴더 내 파일 개수 + 1로 설정
    
    # 새로운 Folder 생성
    os.makedirs(new_path, exist_ok = True)

    # 기본 File Name 설정
    base_filename = df[df['Link'] == link]['Title'].values[0]
    extension = ".png"
    name_index = 1  # File Name에 추가될 숫자
    new_filename = base_filename + extension

    # File Path 생성
    screenshot_name = os.path.join(new_path, new_filename)

    # 동일한 File Name이 존재하는지 확인하고, 존재한다면 새로운 File Name 생성
    while os.path.exists(screenshot_name) :
        new_filename = f"{base_filename}_{name_index}{extension}"
        screenshot_name = os.path.join(new_path, new_filename)
        name_index += 1

    # ScreenShot 촬영 전에 시간 두기. (Loading이 느릴수도 있으므로...)
    time.sleep(3)
    
    # Browser 최대화
    driver.maximize_window()
    
    # 현재 화면 Capture하기
    driver.save_screenshot(screenshot_name)
    # **************************************************************************

    html = driver.page_source
    article_soup = BeautifulSoup(html, "html.parser")
    content = article_soup.select_one("#contents")
    if content :
        # 공백과 HTML Tag 제거
        text = ' '.join(content.text.split())
        articles.append(text)
    # 요청 후 임의의 시간만큼 대기 (Page Loaded)
    time.sleep(random.uniform(0.5, 2.0))

# Browser 종료 (모든 Tab 종료)
driver.quit()

# 빈 문서가 있는지 확인
articles = [article for article in articles if article.strip()]

# 기사 내용이 제대로 수집되었는지 확인
for i, article in enumerate(articles) :
    print(f"기사 {i + 1}: {article[:100]}...")  # 기사 내용 앞부분만 출력

# NLTK를 이용하여 불용어 제거, 단어 토큰화, 표제어 추출
# ****************************************************************
## 한국어 불용어 모음집 불러오기
stopword_list = pd.read_csv("Data/stopword.txt", header = None)
# ****************************************************************
stopword_list[0] = stopword_list[0].apply(lambda x: x.strip())
korean_stopwords = stopword_list[0].to_numpy()

nltk.download('punkt')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()  # 단어의 기본형을 찾기 위해 사용
tokenized_articles = []
"""
    각 기사를 단어 단위로 쪼개고 불필요한 단어들을 제거한 후,
    "tokenized_articles" List에 저장해요.
"""
for article in tqdm(articles, desc = "Text 처리 중", unit = "기사") :
    tokens = word_tokenize(article)
    tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens if word.isalnum() and word.lower() not in korean_stopwords]
    tokenized_articles.append(' '.join(tokens))

# 빈 문서가 있는지 다시 확인
tokenized_articles = [article for article in tokenized_articles if article.strip()]

# 각 기사별 Keyword 추출 (빈도 높은 단어)
keywords = []
for article in tokenized_articles :
    word_counts = Counter(article.split())
    common_words = word_counts.most_common(10)  # 상위 10개 단어 추출
    keywords.append([word for word, freq in common_words])

# Keyword 확인
for i, kw in enumerate(keywords) :
    print(f"기사 {i + 1} 키워드: {kw}")

Crawling 진행 중: 100%|██████████| 32/32 [07:32<00:00, 14.14s/Link]
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/gwangyeong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/gwangyeong/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


기사 1: ▲ 일러스트/한규빛삼척 정라삼거리 인근에서 보행자가 1t 트럭에 치여 숨지는 사고가 발생했다.13일 경찰과 소방 당국 등에 따르면 전날 오후 7시 51분쯤 강원 삼척시 정상동 정라...
기사 2: ▲ 지난 12일 오후 서울 종로구 광화문 건널목에서 시민들이 작열하는 태양 아래 이동하고 있다. 연합뉴스목요일인 13일 전국 대부분 지역에서 30도 이상으로 오르는 곳이 많아 덥겠...
기사 3: ▲ 고성군(군수 함명준)과 동진글로벌씨앤씨(회장 서강연)는 12일 군청 본관에서 군청 부서관계자들과 아야진 해변 상가번영회 회원들, 사업관계자 등 50여명이 참석한 가운데 '아야진...
기사 4: 도소매·숙박음식점업 '찬바람'업계 취업자 수 1년새 13%↓"상환 유예" 정부 정책지원 호소 "월 이자 23% 카드 단기 대출로 버티고 있어요."춘천에서 30년 째 장사하는 윤 모...
기사 5: Video will play after Ad Next subject author Cancel "잠 못 이루는 부안".. 집 들어가기도 무서워 전주MBC 뉴스 Play 1.7K 0:...
기사 6: 인천국제공항 청사 안에서 테니스를 치고 있는 남녀. '보배드림' 인스타그램인천국제공항 청사 안에서 테니스를 치고 있는 한 커플의 영상이 온라인상에서 논란이 되고 있다.지난 12일 ...
기사 7: 엘리베이터 문을 발로 찬 입주민이 게재한 사과문. 온라인 커뮤니티엘리베이터 문이 빨리 닫히지 않아 발로 차 고장 낸 입주민이 780만원의 수리비용을 청구받았다. 그러나 이 입주민은...
기사 8: 집단 휴진에 분노하는 환자들세브란스도 27일부터 무기한 휴진40개 의대 ‘전면 휴진 동참’ 투표“의사가 아파봤다면 이럴 순 없어” 12일 오전 서울 종로구 서울대병원 앞에서 열린 ...
기사 9: 연합뉴스 TV 보도, 헬스장 사장 "어쩔 수 없는 선택""탈의실에서 1~2시간씩 빨래, 드라이기 등 비품 절도" JTBC 사건반장 캡처최근 헬스장 출입문에 '아줌마 출입 금지'라는...
기사 10: 군인권

Text 처리 중: 100%|██████████| 32/32 [00:01<00:00, 26.86기사/s]

기사 1 키워드: ['구독', '기자', '닫기', '강원도민일보', '있습니다', '언론사', '최경진', '기사를', '수', '바로가기']
기사 2 키워드: ['구독', '33도', '강릉', '기자', '닫기', '강원도민일보', '있습니다', '언론사', '0', '서울']
기사 3 키워드: ['아야진', '호텔', '군청', '리조트', '수', '구독', '민간', '군수', '함명준', '회장']
기사 4 키워드: ['구독', '수', '지난해', '닫기', '기자', '강원도민일보', '있습니다', '언론사', '이자', '대출']
기사 5 키워드: ['구독', '전주mbc', '뉴스', '기자', '닫기', 'subtitle', '수', '지진', '있습니다', '언론사']
기사 6 키워드: ['구독', '테니스를', '치고', '있는', '기자', '국민일보', '닫기', '있습니다', '언론사', '청사']
기사 7 키워드: ['발로', '구독', '문을', '고', '고장', '며', '입주민은', '후', '맨발로', '수']
기사 8 키워드: ['구독', '기자', '휴진에', '휴진', '고', '국민일보', '무기한', '오는', '이날', '병원을']
기사 9 키워드: ['수', '구독', '닫기', '매일신문', '고', '헬스장', '기자', '볼', '있습니다', '언론사']
기사 10 키워드: ['구독', '닫기', '매일신문', '수', '중대장이', '기자', '있습니다', '언론사', '군인권센터', '많은']
기사 11 키워드: ['이재명', '한동훈', '구독', '조국', 'p', '닫기', '매일신문', '이준석', '홍준표', '은']
기사 12 키워드: ['수', '구독', '닫기', '매일신문', '청약부금', '기자', '볼', '있습니다', '언론사', '청약통장']
기사 13 키워드: ['구독', '수', '있습니다', '닫기', 'jib', '언론사', '뉴스', '피해를', '차독나방이', '기

> base_path 설정 부분: <br>
os.path.exists(file_path) 조건을 사용하여 실제 경로가 존재하는지 확인합니다. <br>
경로가 존재하면 file_path에 새로운 폴더 번호를 추가하여 경로를 생성합니다.

> new_filename 생성 부분: <br>
os.path.exists(screenshot_name) 조건을 사용하여 실제 파일 경로가 존재하는지 확인합니다. <br>
파일이 존재하면 new_filename에 새로운 번호를 추가하여 파일 이름을 생성합니다.

> os.path.join() 함수는 운영 체제에 맞는 경로 구분자(Windows의 '', Unix/Linux의 '/')를 자동으로 처리해주기 때문에, <br>
상대경로나 절대경로를 모두 안전하게 연결할 수 있습니다.

---

## Deep Learning
- TF-IDF 벡터화 <br><br>
- 코사인 유사도 <br><br>
- <span style = "color: #F7CAC9">**오늘의 주요 키워드에 따른 뉴스 추천하기**</span>

### TF-IDF 벡터화

In [20]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(tokenized_articles)

> TfidfVectorizer를 사용해 각 기사를 숫자로 표현해요. <br><br>
이를 통해 컴퓨터가 기사를 이해할 수 있게 돼요.

### 코사인 유사도

In [21]:
cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

> cosine_similarity를 사용해 각 기사 간의 유사도를 계산해요.

### 추천할 뉴스 인덱스 추출

In [22]:
top_news_indices = []
for i in tqdm(range(len(cosine_similarities)), desc = "유사도 계산 중", unit = "기사"):
  similar_indices = cosine_similarities[i].argsort()[:-11:-1]
  top_news_indices.append(similar_indices)

유사도 계산 중: 100%|██████████| 32/32 [00:00<00:00, 33222.21기사/s]


> 각 기사에 대해 유사도가 높은 상위 10개의 기사를 찾고, top_news_indices 리스트에 저장해요.

### 추천할 뉴스 출력

In [23]:
for i, indices in enumerate(top_news_indices) :
  print(f"\n--- 추천 뉴스 {i + 1} ---")
  for j, idx in enumerate(indices) :
    if idx != i :
      print(f"{j + 1}. {links[idx]}")


--- 추천 뉴스 1 ---
2. https://n.news.naver.com/article/654/0000077708?ntype=RANKING
3. https://n.news.naver.com/article/654/0000077681?ntype=RANKING
4. https://n.news.naver.com/article/661/0000041296?ntype=RANKING
5. https://n.news.naver.com/article/661/0000041294?ntype=RANKING
6. https://n.news.naver.com/article/655/0000019014?ntype=RANKING
7. https://n.news.naver.com/article/654/0000077596?ntype=RANKING
8. https://n.news.naver.com/article/661/0000041267?ntype=RANKING
9. https://n.news.naver.com/article/002/0002335982?ntype=RANKING
10. https://n.news.naver.com/article/005/0001703187?ntype=RANKING

--- 추천 뉴스 2 ---
2. https://n.news.naver.com/article/654/0000077710?ntype=RANKING
3. https://n.news.naver.com/article/654/0000077681?ntype=RANKING
4. https://n.news.naver.com/article/654/0000077596?ntype=RANKING
5. https://n.news.naver.com/article/661/0000041294?ntype=RANKING
6. https://n.news.naver.com/article/655/0000019014?ntype=RANKING
7. https://n.news.naver.com/article/002/0002335982?ntyp

> 각 기사에 대해 추천할 뉴스 기사를 출력해요. <br><br>
자기 자신은 제외해요.